# Relation extraction

Previously:

* Introduction to Information Extraction
* Named Entity Recognition
* Normalization and Entity linking

---

**Named Entity Recognition** (NER) recognizes mentions of target entities in text and (typically) assign each a type (e.g. `PERSON`, `LOCATION`)

<img width="90%" src="https://raw.githubusercontent.com/TurkuNLP/turku-ner-corpus/master/docs/example.png">

---

**Entity linking** identifies the real-word entities referred to in text by associating mentions with IDs in a knowledge base (e.g. Wikidata):

<img width="90%" src="https://github.com/TurkuNLP/Text_Mining_Course/raw/master/figs/entity-linking-marin.png">

---

Here, we will assume that NER and normalization have already been performed and build on their outputs.

**Relation extraction** aims to identify meaninful associations between mentions of (named) entities. The associations are represented as relations, which often (but not necessarily) also have a type.

We previously discussed some relations found in Wikidata, such as

* `instance_of(Douglas Adams, human)`
* `date_of_birth(Douglas Adams, 11 March 1952)`
* `country_of_citizenship(Douglas Adams, United Kingdom)`

These are _typed, binary_ relations: each relation has a type (e.g. `country_of_citizenship`) and relates exactly two things (e.g. `Douglas Adams` and `United Kingdom`). These relations (and more) could be extracted from the following text:

<blockquote>
Douglas Noel Adams (11 March 1952 – 11 May 2001) was an English author
</blockquote>

As was noted at in the material on [Normalization](https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/Normalization.ipynb), relation extraction allows us to populate knowledge bases with information discovered from text.

**Examples of relation extraction targets** include

* Part-whole: `part_of(Cambridge, United Kingdom)`
* Location: `place_of_birth(Douglas Adams, Cambridge)`
* Family: `father(Douglas Adams, Christopher Adams)`
* Membership: `member(Clarence Thomas, Supreme Court)`
* Affiliation: `employee(Larry Page, Google)`
* Ownership: `owner(John Smith, John's Hardware store)`
* ... and many, many more (see e.g. [ACE relations](https://www.ldc.upenn.edu/sites/www.ldc.upenn.edu/files/english-relations-guidelines-v6.2.pdf))

In specialized domains, relation extraction can target e.g.

* Protein-protein interactions
* Drug-disease relations
* Drug side-effects
* Organism-habitat relations
* etc.

## Task setting

We could potentially target relations of any _arity_, for example

* ternary: `parents(Douglas Adams, Janet Adams, Christopher Adams)`
* ternary: `date_and_place_of_birth(Douglas Adams, 11 March 1952, Cambridge)`
* 4-ary: `dates_of_birth_and_death_and_nationality(Douglas Adams, 11 March 1952, 11 May 2001, United Kingdom)`

For simplicity, we will here focus on binary relations, also noting that many ternary, 4-ary etc. relations can be decomposed into binary relations.

Let's first consider the following generalized task setting for untyped binary relation extraction:

* Given a set of text documents $D = \{ d_1, d_2, \ldots d_n \}$, and
* For each document $d$, given mentions of entities in that document $M_d = \{m_1, m_2, \ldots, m_n \}$
* Return for each document relations $R_d \subset M_d \times M_d$ that hold between the entities based on $d$

So, here a relation is represented as simply a pair of mentions.

We will here assume that the mentions $m$ are identified by start and end offsets, type, and text, so that for example if

$d =$ `Douglas Adams is an English author ...`

we could have

$M_d = \{$ `(0, 13, "PERSON", "Douglas Adams")`, `(20, 27, "NORP", "English")`, $\ldots \}$

in code, let's define

In [1]:
from collections import namedtuple


Mention = namedtuple('Mention', 'start end type text')


# Example document and entities
d = 'Douglas Adams is an English author'
M_d = [
  Mention(0, 13, 'PERSON', 'Douglas Adams'),
  Mention(20, 27, 'GPE', 'English'),
]

# Note that mention text is redundant given text and the (start, end) offsets.
# The mention text is included here only for convenience.
for m in M_d:
  print(m, '→', d[m.start:m.end])

Mention(start=0, end=13, type='PERSON', text='Douglas Adams') → Douglas Adams
Mention(start=20, end=27, type='GPE', text='English') → English


(Note that we're not explicitly including normalization results here, but will assume normalization to be available when required.)

## Metrics

As most relations in $M_d \times M_d$ (i.e. all combinations of entity pairs in a document) do not hold, a trivial "method" that never returns any relations could have deceptively high _accuracy_: for example, if there are 100 possible pairs in a document and a relation holds for 5, returning an empty set of relations would give 95% accuracy.

For this reason, accuracy is rarely used to evaluate relation extraction; instead, the standard information retrieval metrics are used:

* **Precision**: the ratio of extracted relations that are correct
* **Recall**: the ratio of correct relations that are extracted
* **$F_1$-score**: the balanced harmonic mean of precision and recall

(These metrics should already be familiar from previous material.)

## Relation extraction approaches

There are several approaches we can take to relation extraction, including

* **Co-occurrences**: assume that mentions occurring in the same context have some relation
* **Rules**: write explicit instructions (code) for extracting relations
* **Machine learning**: annotate example relations and train ML method to extract them

We'll cover each of these in this session.

---

# Co-occurrence-based relation extraction

Relation extraction based primarily on the assumption that when entity mentions **occur together** in some context (e.g. sentence, paragraph, document), some relation is likely to hold between them.

The simplest form of co-occurrence based relation extraction would be to always return a relation for all pairs of co-occurring entities. Let's illustrate this for a very simple sentence:

In [2]:
import spacy


def cooccurrence_relations(mentions):
  relations = []
  for i, mention1 in enumerate(mentions):
    for j, mention2 in enumerate(mentions[i+1:]):
      relations.append((mention1, mention2))
  return relations


text = 'Douglas Adams (born March 11th 1952) was a British author.'

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for m1, m2 in cooccurrence_relations(doc.ents):
  print((m1.text, m2.text))

('Douglas Adams', 'March 11th 1952')
('Douglas Adams', 'British')
('March 11th 1952', 'British')


For this simple example, we get two valid relations:

* (Douglas Adams, March 11th 1952)
* (Douglas Adams, British)

and one erroneous (or meaningless) relation:

* (March 11th 1952, British)

In general, the naive co-occurrence based approach has perfect recall (all relevant relations are included) but typically very poor precision: most of the returned relations are wrong. This is evident already with slightly longer text:

In [3]:
text = [
  'Adams was born on 11 March 1952 to Janet and Christopher Adams in Cambridge.',
  'The family later moved to London, where his sister, Susan, was born.',
]

doc = nlp(' '.join(text))
for m1, m2 in cooccurrence_relations(doc.ents):
  print((m1.text, m2.text))

('Adams', '11 March 1952')
('Adams', 'Janet')
('Adams', 'Christopher Adams')
('Adams', 'Cambridge')
('Adams', 'London')
('Adams', 'Susan')
('11 March 1952', 'Janet')
('11 March 1952', 'Christopher Adams')
('11 March 1952', 'Cambridge')
('11 March 1952', 'London')
('11 March 1952', 'Susan')
('Janet', 'Christopher Adams')
('Janet', 'Cambridge')
('Janet', 'London')
('Janet', 'Susan')
('Christopher Adams', 'Cambridge')
('Christopher Adams', 'London')
('Christopher Adams', 'Susan')
('Cambridge', 'London')
('Cambridge', 'Susan')
('London', 'Susan')


The combinatorial explosion of wrong or irrelevant pairs arising from naive co-occurrence based relation extraction can be alleviated by restricting the scope of considered co-occurrences to e.g. sentences, but this does compromise on the perfect recall of the approach: relations can involve entities mentioned in different sentences.

Note additionally that this approach does not provide any information about the _types_ of the relations.

## Statistical approaches

Co-occurrence provides only weak evidence that the co-mentioned entities are associated. However, by accumulating such evidence over a large corpus, it is possible to identify associations with higher confidence than a single co-occurrence can give.

Let's illustrate this by looking at mentions tagged in a sample of English Wikipedia articles.

In [4]:
!wget -nc https://a3s.fi/TKO_8964_2021/en-wiki-sample-ontonotes.conll.gz

--2021-04-19 05:28:38--  https://a3s.fi/TKO_8964_2021/en-wiki-sample-ontonotes.conll.gz
Resolving a3s.fi (a3s.fi)... 86.50.254.18, 86.50.254.19
Connecting to a3s.fi (a3s.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79442456 (76M) [application/gzip]
Saving to: ‘en-wiki-sample-ontonotes.conll.gz’

en-wiki-sample-onto 100%[===================>]  75.76M  68.7MB/s    in 1.1s    

2021-04-19 05:28:39 (68.7 MB/s) - ‘en-wiki-sample-ontonotes.conll.gz’ saved [79442456/79442456]



We'll use a reader function for the two-column (text, tag) CoNLL format. You should already be familiar with this representation from previous material.

In [5]:
import gzip


def read_conll_entities(stream):
  words, tags = [], []
  for ln, line in enumerate(stream, start=1):
    if line.startswith('#'):
      continue    # skip comments
    elif line.isspace():
      if words and tags:
        yield words, tags
      words, tags = [], []
    else:
      word, tag = line.rstrip('\n').split('\t')
      words.append(word)
      tags.append(tag)
  if words and tags:
    yield words, tags


conll_sentences = []
with gzip.open('en-wiki-sample-ontonotes.conll.gz', 'rt', encoding='utf-8') as f:
  for words, tags in read_conll_entities(f):
    conll_sentences.append((words, tags))

This NER annotation is represented in the familiar IOB (in-out-begin) format (see [NER notebook](https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/NER_introduction.ipynb)):

In [6]:
for word, tag in zip(*conll_sentences[1]):
  print(f'{word}\t{tag}')

Ayn	B-PERSON
Rand	I-PERSON
(	O
;	O
born	O
Alisa	B-PERSON
Zinovyevna	I-PERSON
Rosenbaum	I-PERSON
;	O
–	O
March	B-DATE
6	I-DATE
,	I-DATE
1982	I-DATE
)	O
was	O
a	O
Russian	B-NORP
-	I-NORP
American	I-NORP
writer	O
and	O
philosopher	O
.	O


Let's next convert these IOB-tagged sentences into the mention format defined above, where each mention is defined by its start and end offsets, type, and text. (You don't need to understand the conversion steps in detail.)

In [7]:
def conll_to_mentions(words, tags):
  mentions, offset, start, label = [], 0, None, None
  sentence = ' '.join(words)
  for word, tag in zip(words, tags):
    if tag[0] in 'OB' and start is not None:    # current ends
      end = offset-1
      mentions.append(Mention(start, end, label, sentence[start:end]))
      start, label = None, None
    if tag[0] == 'B':
      start, label = offset, tag[2:]
    elif tag[0] == 'I':
      if start is None:    # I without B, but nevermind
        start, label = offset, tag[2:]
    else:
      assert tag == 'O', 'unexpected tag {}'.format(tag)
    offset += len(word) + 1    # +1 for space
  if start is not None:    # span open at sentence end
    end = offset-1
    mentions.append(Mention(start, end, label, sentence[start:end]))
  return mentions


sentences = []
mentions_by_sentence = []
for words, tags in conll_sentences:
  sentences.append(' '.join(words))
  mentions_by_sentence.append(conll_to_mentions(words, tags))

Now we have for the above example sentence

In [8]:
for mention in mentions_by_sentence[1]:
  print(mention)

Mention(start=0, end=8, type='PERSON', text='Ayn Rand')
Mention(start=18, end=44, type='PERSON', text='Alisa Zinovyevna Rosenbaum')
Mention(start=49, end=63, type='DATE', text='March 6 , 1982')
Mention(start=72, end=90, type='NORP', text='Russian - American')


This is the `(start, end, type, text)` representation that we want for relation extraction.

Let's next see if we can find anything of interest just by looking at the most common co-occurrences.

In [9]:
from collections import Counter


relation_counts = Counter()
for mentions in mentions_by_sentence:
  for relation in cooccurrence_relations(mentions):
    relation_counts[relation] += 1

for relation, count in relation_counts.most_common(10):
  m1, m2 = relation[0], relation[1]
  print(f'{count}\t({m1.text}/{m1.type}, {m2.text}/{m2.type})')

3939	(every 100/CARDINAL, age 18/DATE)
3738	(Hispanic/NORP, Latino/NORP)
1177	(25 to 44/DATE, 45 to 64/DATE)
1095	(the age of 18/DATE, 45 to 64/DATE)
1087	(the age of 18/DATE, 25 to 44/DATE)
1020	(the age of 18/DATE, 65 years of age or older/DATE)
999	(45 to 64/DATE, 65 years of age or older/DATE)
986	(25 to 44/DATE, 65 years of age or older/DATE)
816	(18/CARDINAL, 24/CARDINAL)
807	(24/CARDINAL, 45 to 64/DATE)


These aren't really interesting; most of the pairs involve a pair of numeric mention types. On reflection, it seems reasonable to assume that at least one of each pair of potentially related entities should have something other than a numeric type. We might also get more relevant associations by focusing further on specific types. Let's try this out.

This data uses [OntoNotes types](https://catalog.ldc.upenn.edu/docs/LDC2013T19/OntoNotes-Release-5.0.pdf#page=21):

In [10]:
ALL_TYPES = set(m.type for mentions in mentions_by_sentence for m in mentions)

ALL_TYPES

{'CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART'}

Let's see if we get more interesting pairings by filtering by type:

In [11]:
def filtered_cooccurrence_relations(mentions, types1, types2):
  relations = []
  for i, mention1 in enumerate(mentions):
    for mention2 in mentions[i+1:]:
      if (mention1.type in types1 and mention2.type in types2 and
          mention1.type != mention2.type):
        relations.append((mention1, mention2))
  return relations


# Try changing the following to other types! (see above for list)
TYPES1 = { 'PERSON' }
TYPES2 = { 'ORG' }

relation_counts = Counter()
for mentions in mentions_by_sentence:
  for relation in filtered_cooccurrence_relations(mentions, TYPES1, TYPES2):
    relation_counts[relation] += 1

for relation, count in relation_counts.most_common(20):
  m1, m2 = relation[0], relation[1]
  print(f'{count}\t({m1.text}/{m1.type}, {m2.text}/{m2.type})')

9	(Bosley Crowther/PERSON, `` The New York Times/ORG)
8	(Vincent Canby/PERSON, The New York Times/ORG)
8	(Janet Maslin/PERSON, The New York Times/ORG)
7	(Gene Siskel/PERSON, the `` Chicago Tribune/ORG)
6	(Charles Champlin/PERSON, the `` Los Angeles Times/ORG)
4	(Roger Ebert/PERSON, Chicago Sun-Times/ORG)
3	(Gary Arnold/PERSON, The Washington Post/ORG)
3	(A. O. Scott/PERSON, The New York Times/ORG)
3	(St John 's/PERSON, College/ORG)
3	(A. H. Weiler/PERSON, The New York Times/ORG)
3	(Häkkinen/PERSON, McLaren/ORG)
3	(Tuoba Xianbei/PERSON, the Northern Wei/ORG)
2	(Stephen Thomas Erlewine/PERSON, AllMusic/ORG)
2	(Martin Richards/PERSON, the University of Cambridge/ORG)
2	(Charles II/PERSON, the Hudson 's Bay Company/ORG)
2	(Charles II/PERSON, HBC/ORG)
2	(Cranmer/PERSON, Church/ORG)
2	(Cranmer/PERSON, Prayer Books/ORG)
2	(Roosevelt/PERSON, Congress/ORG)
2	(Mick LaSalle/PERSON, the `` San Francisco Chronicle/ORG)


That's better, there are definitely valid relations there that could potentially serve as entries in a knowledge base given normalization (and a relation type!)

In this simple statistical approach, we have ranked the relations by their total count in the corpus. While this simple ranking approach can work, it should be noted that it fails to account for the probability of chance co-occurrence.

An improved statistical approach would be to use the overall frequency of occurrence in the data to get an estimate of the expected number of co-occurrences for each pair, and divide the counts of actual co-occurrences by this number to determine which pairs co-occur more frequently than expected by chance.

---

# Rule-based relation extraction

Broadly, any method involving explicitly written instructions for how to extract relations. (Contrast with machine learning approaches, where the "rules" are learned from examples.)

Let's first consider a simple extension of naive co-occurrence based relation extraction to assign candidate types to co-occurring mentions by looking for keywords in their context -- here, the words in between the mentions.



In [12]:
relation_keywords = {
    'date_of_birth': ['born', 'birth'],
    'date_of_death': ['died', 'dead', 'killed'],
}

relation_counts = Counter()
for sentence, mentions in zip(sentences, mentions_by_sentence):
  for i, mention1 in enumerate(mentions):
    for mention2 in mentions[i+1:]:
      if mention1.type == 'PERSON' and mention2.type == 'DATE':
        between_words = sentence[mention1.end:mention2.start].split()
        for rel, keywords in relation_keywords.items():
          if any(k in between_words for k in keywords):
            relation_counts[(rel, mention1.text, mention2.text)] += 1

for relation, count in relation_counts.most_common(20):
  rel, m1, m2 = relation[0], relation[1], relation[2]
  print(f'{count}\t{rel}({m1}, {m2})')

4	date_of_death(Richard Montgomery, 1775)
3	date_of_birth(Afonso, 1109)
3	date_of_death(Queen Victoria, 22 January 1901)
3	date_of_death(Garret Hobart, 1899)
2	date_of_death(Afonso, 1185)
2	date_of_death(Isabella, 1103)
2	date_of_death(Peter, 1103)
2	date_of_birth(Mozart, about four months)
2	date_of_death(Gur, October 1967)
2	date_of_death(Gur, 11)
2	date_of_birth(Britney Jean Spears, December 2 , 1981)
2	date_of_birth(Billy Bob Thornton, August 4 , 1955)
2	date_of_death(DeMille, November 1959)
2	date_of_birth(David Andrew, August 28 , 1962)
2	date_of_birth(Leo Fincher, August 28 , 1962)
2	date_of_birth(Denis Colin Leary, 18 August 1957)
2	date_of_birth(Eve Arden, April 30 , 1908)
2	date_of_death(Franco, 1975)
2	date_of_death(Louis XVIII, September 1824)
2	date_of_death(Charles II, 1700)


There's a lot of noise here, but at least the following relation extracted for [Queen Victoria](https://en.wikipedia.org/wiki/Queen_Victoria) appears to hold:

```
date_of_death(Queen Victoria, 22 January 1901)
```

Like co-occurrence based extraction, this simple approach suffers from low precision. To improve on this, let's look at an important subclass of rule-based relation extraction methods, ones that involve explicit **patterns** for relation extraction.

Extraction patterns can be expressed over the linear sequence of words ("flat" patterns) or over some representation of syntactic structure (e.g. dependency patterns). We'll look into both in the following.


## Flat pattern matching

Flat pattern matching can be implemented using standard tools such as [regular expressions](https://en.wikipedia.org/wiki/Regular_expression). (If you're not familiar with regular expression syntax or need a refresher, you may want to have a look at e.g. the detailed [regular expression HOWTO](https://docs.python.org/3/howto/regex.html) included in Python documentation.)

Let's first briefly revisit the extraction of is-a relations using [Hearst patterns](https://people.ischool.berkeley.edu/~hearst/papers/coling92.pdf). The basic idea is that we can infer _is-a_ (or _type-of_, or [_hyponym_](https://en.wikipedia.org/wiki/Hyponymy_and_hypernymy)) relations by looking for patterns such as the following:

```
authors such as Shakespeare    →  is-a(author, Shakespeare)
libraries and other buildings  →  is-a(building, library)
countries, including Finland   →  is-a(country, Finland)
```

Let's try a "such as" pattern on our example Wikipedia data. We're here fixing one participant in the relation to be an entity mention, and simply matching the other as a string using a regular expression.

In [13]:
import re


# Explanation for regular expression:
# \b         matches word boundary
# [a-z]+s    matches any sequence of "a" to "z" characters, ending with "s"
# ( ,)?      matches optional comma
# such as $  matches the text "such as" at the end of the string
SUCH_AS_RE = re.compile(r'\b([a-z]+s)( ,)? such as $')


relation_counts = Counter()
for sentence, mentions in zip(sentences, mentions_by_sentence):
  for mention2 in mentions:
    before = sentence[:mention2.start]
    m = SUCH_AS_RE.search(before) 
    if m:
      mention1 = m.group(1)
      relation = ('is-a', mention1, mention2.text)
      relation_counts[relation] += 1


for relation, count in relation_counts.most_common(10):
  type_, m1, m2 = relation[0], relation[1], relation[2]
  print(f'{count}:\t{type_}({m1}, {m2})')

8:	is-a(countries, the United States)
8:	is-a(countries, China)
5:	is-a(languages, Hindi)
5:	is-a(countries, India)
5:	is-a(countries, Japan)
5:	is-a(countries, France)
4:	is-a(languages, French)
4:	is-a(languages, English)
4:	is-a(countries, the United Kingdom)
4:	is-a(countries, Germany)


Not bad, but those plural forms really don't work here. Let's add a quick fix to lemmatize these:

In [14]:
import nltk
from nltk.stem import WordNetLemmatizer 


nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [15]:
lemmatizer = WordNetLemmatizer()

for relation, count in relation_counts.most_common(20):
  type_, m1, m2 = relation[0], relation[1], relation[2]
  m1 = lemmatizer.lemmatize(m1)
  print(f'{count}:\t{type_}({m1}, {m2})')

8:	is-a(country, the United States)
8:	is-a(country, China)
5:	is-a(language, Hindi)
5:	is-a(country, India)
5:	is-a(country, Japan)
5:	is-a(country, France)
4:	is-a(language, French)
4:	is-a(language, English)
4:	is-a(country, the United Kingdom)
4:	is-a(country, Germany)
3:	is-a(power, France)
3:	is-a(country, Australia)
3:	is-a(city, London)
3:	is-a(country, Egypt)
3:	is-a(country, Saudi Arabia)
3:	is-a(band, Korn)
3:	is-a(language, Haskell)
3:	is-a(city, Bukhara)
3:	is-a(language, Sanskrit)
2:	is-a(city, Athens)


Or, alternatively, we can group these by hypernym:

In [16]:
from collections import defaultdict


names_by_hypernym = defaultdict(list)
for relation in relation_counts:
  type_, m1, m2 = relation[0], relation[1], relation[2]
  names_by_hypernym[m1].append(m2)

# only print longer lists to avoid excessive output
for m1, m2_list in names_by_hypernym.items():
  if len(m2_list) > 25:
    print(f'{m1:14s}:', ', '.join(m2_list))

writers       : Erika Holzer, Hesiod, Francis Bacon 's, Richard Hakluyt, Edward A. Pollard, Lactantius, Frank Báez, Virgil, Gail Simone, Hippolytus, Ursula K. Le Guin, James Fordyce, the 17th - century, Allen Ginsberg, Constance Cumbey, H. G. Wells, Goldstücker, Alexander Pope, Ahmad Shamlou, Giovanni Villani, Machiavelli, William S. Burroughs, Clark Ashton Smith, Saint Jerome, Michael Field, Elizabeth Harrower, Beaumont, Glenn Reynolds, Stephen Crane, Paul Huson, Alejo Carpentier, Salman Rushdie, Albert Camus, Moritz August von Thümmel, Lao She, Plutarch, Remarque, Helen Stevenson, Robert E. Howard, Emile Danoën, Stephen White, Alan Watts, Kerouac, Miguel de Cervantes, Samuel Presbiter, Mathew, Kristine Kathryn Rusch, Isaac Asimov, Richard Dawkins, Anna Seward, Olive Schreiner, John McGahern, Humfrey Barwick
languages     : Hindi, Eiffel, Python, Jalapa Mazatec, French, English, Catalan, JavaScript, CSS, Spanish, Hausa, Sindhi, Banda, Serui, Nicaraguan Sign Language, Korean, Latin, Ar

Note that the relations above are still expressed in terms of strings, not IDs representing the relevant entities in a knowledgebase. 

To have fully structured relations, we would need to associate the strings such as "author" with knowledge base IDs such as Wikidata [Q482980](https://www.wikidata.org/wiki/Q482980) and make use of normalization information for the named entity mention.

## Dependency pattern matching

The "flat" Hearst patterns succeed in part because the expressions that they match have limited syntactic variability. Let's look at a relation that is commonly expressed in more varied ways, finding strings containing "married" between two `PERSON` mentions:

In [17]:
between_counts = Counter()
for sentence, mentions in zip(sentences, mentions_by_sentence):
  for i, mention1 in enumerate(mentions):
    for j, mention2 in enumerate(mentions[i+1:]):
      if mention1.type == 'PERSON' and mention2.type == 'PERSON':
        between = sentence[mention1.end:mention2.start]
        if 'married' in between:
          between_counts[between] += 1


for between, count in between_counts.most_common(10):
  print(f'{count}\tPERSON1{between}PERSON2')

214	PERSON1 married PERSON2
19	PERSON1 , married PERSON2
16	PERSON1 was married to PERSON2
7	PERSON1 married his first wife , PERSON2
6	PERSON1 had married PERSON2
6	PERSON1 , who married PERSON2
5	PERSON1 is married to PERSON2
5	PERSON1 married his second wife , PERSON2
5	PERSON1 , married to PERSON2
4	PERSON1 married actress PERSON2


Here we see much more variability in the linear word sequence. However, if we instead look at dependency analyses for these sentences: (try the [parser demo](http://bionlp-www.utu.fi/parser_demo/) live!):

<img width="65%" src="https://github.com/TurkuNLP/Text_Mining_Course/raw/master/figs/john-married-jane.png">

We see that the _dependency paths_ connecting the two named entities are much more regular:

```
John  ←      nsubj ← married → obj → Jane
John  ← nsubj:pass ← married → obj → Jane
John  ← nsubj:pass ← married → obj → Jane
John  ←      nsubj ← married → obj → wife → appos → Jane
```

(The definitions for the dependency types can be found in the [Universal Dependencies](https://universaldependencies.org/) documentation: [nsubj](https://universaldependencies.org/u/dep/nsubj.html), [nsubj:pass](https://universaldependencies.org/u/dep/nsubj-pass.html), [obj](https://universaldependencies.org/u/dep/obj.html), [appos](https://universaldependencies.org/u/dep/appos.html))

The observation that the _shortest dependency path_ connecting two entity mentions contains most of the information regarding their relation (if any) has inspired [many relation extraction methods](https://scholar.google.com/scholar?q="shortest+dependency+path"+"relation+extraction").

We won't go into full code here, but we can sketch a dependency pattern-based relation extraction approach as follows:

---

* Write extraction patterns over dependency paths
* For each sentence $s$:
    * Parse sentence $s$, producing dependency graph $d$
    * For each pair of entity mentions in the sentence $(m_1, m_2)$:
        * Find the shortest path in $d$ connecting $m_1$ to $m_2$
        * Linearize the shortest path (e.g. to `ENTITY1 ← nsubj ← married → obj → ENTITY2`)
        * Match each pattern against the dependency path representation and return a relation for each pattern that matches

---

Note that because dependency paths can be linearized to strings, we can use standard methods such as regular expressions to match patterns also against dependency paths. Alternatively, we could use dedicated tools such as [Tregex](https://nlp.stanford.edu/software/tregex.shtml).

## Open Information Extraction

The rule-based approaches considered above have required us to specify the relation types in advance, e.g. through keywords or patters for each type. By contrast, Open Information Extraction (Open IE) methods aim to identify _any_ relation stated in text, extracting the relation type along with the participating entities.

Open IE methods can operate over the linear sequence of words (making use of e.g. part-of-speech patterns) or over syntactic structures, such as in the following example from the [Stanford Open IE system](https://nlp.stanford.edu/software/openie.html):

<img width="90%" src="https://nlp.stanford.edu/static/img/openie.png">

(Figure from https://nlp.stanford.edu/software/openie.html)


Open IE is at its best when applied to very large corpora. Instead of trying to run a large-scale implementation in this notebook, let's have a look at the results of the [ReVerb](http://reverb.cs.washington.edu/) Open IE system applied to 500 million web pages on this demo site: https://openie.allenai.org/

While Open IE systems have mostly been developed for English, the introduction of the cross-lingually consistent [Universal Dependencies](https://universaldependencies.org/) resources has made it possible to write Open IE rules in a (largely) language-independent way. [PredPatt](https://github.com/hltcoe/PredPatt) implements one such approach; we'll have a brief practical look at it next.

Install the PredPatt library (with some changes for better support for Finnish)

In [18]:
!pip install --quiet git+https://github.com/spyysalo/PredPatt.git

     |████████████████████████████████| 368kB 11.0MB/s 
     |████████████████████████████████| 92kB 5.2MB/s 
     |████████████████████████████████| 460kB 22.7MB/s 
     |████████████████████████████████| 1.4MB 27.2MB/s 
     |████████████████████████████████| 92kB 6.8MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 81kB 6.2MB/s 
     |████████████████████████████████| 10.1MB 35.7MB/s 


Add a `parse_sentence` function using the [Turku Neural parser](http://turkunlp.org/Turku-neural-parser-pipeline/) as a web service

In [19]:
!pip install --quiet requests

import requests
from predpatt import load_conllu


def parse_sentence(sentence):
    SERVER_URL = 'http://86.50.253.19:8002/parser/parse'
    response = requests.post(SERVER_URL, data={ 'text': sentence })
    return load_conllu(response.text)

(**NOTE**: if you're running this after the 2021 spring course, this service is likely no longer available at the above URL.)


Parse example sentences using the Turku parser, and extract relations from each parse using PredPatt.

(The technical details are not important here; this is just demonstrating an Open IE approach for Finnish.)

In [20]:
from predpatt import PredPatt, PredPattOpts


# PredPatt options
options = PredPattOpts(
  resolve_relcl=True,
  resolve_appos=True,
  resolve_conj=True,
  ud='2.0'
)


# Function to format PredPatt result as a string
def format_predicate(predicate):
  rel = '-'.join(token.text for token in predicate.tokens)
  args = []
  for argument in predicate.arguments:
    args.append(' '.join(token.text for token in argument.tokens))
  arg_string = ', '.join(args)
  return f'{rel}({arg_string})'


# Try changing these!
example_sentences = [
  'Turun yliopisto perustettiin vuonna 1920.',
  'Turun kampus sijaitsee Yliopistonmäen läheisyydessä.',
  'Turun yliopisto toimii Turussa, Raumalla ja Porissa.',
]


for sentence in example_sentences:
  for id_, parse in parse_sentence(sentence):
    ppatt = PredPatt(parse, opts=options)
    print(sentence)
    for predicate in ppatt.instances:
      print('→', format_predicate(predicate))
    print()

Turun yliopisto perustettiin vuonna 1920.
→ perustettiin(Turun yliopisto, vuonna 1920)

Turun kampus sijaitsee Yliopistonmäen läheisyydessä.
→ sijaitsee(Turun kampus, Yliopistonmäen läheisyydessä)

Turun yliopisto toimii Turussa, Raumalla ja Porissa.
→ toimii(Turun yliopisto, Turussa)
→ toimii(Turun yliopisto, Raumalla)
→ toimii(Turun yliopisto, Porissa)



Note again that these extractions are just strings; to enter the extracted relations into a knowledge base, we would need to map the text spans to normalized entity mentions and relation IDs.

---

# Relation extraction using machine learning

The most accurate relation extraction methods are based on supervised machine learning, i.e. models trained on examples of which relations to extract from documents.

We'll here first look at two options for annotating data for relation extraction, and then sketch a machine learning approach for learning to extract relations.


## Relation annotation

Like all supervised machine learning approaches, ML-based relation extraction methods must be trained on examples of inputs with correct outputs. Recall that in our task setting, the input includes document texts $d$ and entity mentions $M_d = \{m_1, m_2, \ldots, m_n \}$ in each document.

We will then assume that the output consists of typed binary relations $R_d \subset T \times M_d \times M_d$ where $T$ is the set of types. We could for example have $T = \{$ `employee`, `owner`, `founder` $\}$ for person-organization relations.

Using a **document-oriented** annotation approach (here the open-source tool [brat](https://brat.nlplab.org)), our starting point could then look like the following:

<img width="65%" src="https://github.com/TurkuNLP/Text_Mining_Course/raw/master/figs/relation-annotation-empty.png">

We would then read the text of the documents (here, sentences) and mark all applicable relations:

<img width="65%" src="https://github.com/TurkuNLP/Text_Mining_Course/raw/master/figs/relation-annotation-complete.png">

The annotated relations are then positive examples for machine learning:

```
founder(Bill Gates, Microsoft)
founder(Paul Allen, Microsoft)
employee(Steve Ballmer, Microsoft)
```

Here, we have not explicitly marked any _negative_ examples. For training machine learning methods, negatives are instead generated using the _closed world assumption_: any relation that is not explicitly present is assumed to be negative, including e.g.

```
founder(Allen, IBM)
employee(Allen, IBM)
owner(Allen, IBM)
...
founder(Steve Ballmer, Microsoft)
owner(Steve Ballmer, Microsoft)
```

Alternatively, we can take a **relation-oriented** annotation approach (here using [Prodigy](https://prodi.gy/)), where the tool prompts the annotator to identify the type of relation (if any) that holds between two entities:

<img width="65%" src="https://github.com/TurkuNLP/Text_Mining_Course/raw/master/figs/prodigy-typed-relation-annotation.png">

Here negative examples are explicitly marked by the annotator (label `NONE`). While this approach thus requires the annotator to enter a larger number of labels, individual decisions can be faster to enter, and the explicitness can help reduce errors of omission.

This approach can be carried further by incorporating the type of the relation into the prompt, only asking the annotator whether a `(relation, mention-1, mention-2)` triple holds or not. In this case, the decision can be a simple binary yes/no, further accelerating the annotation process.

## Machine learning task formulation

Given manually annotated examples where each consists of

* Document text $d$
* Two entity mentions in that document, $m_1$ and $m_2$
* Relation type (e.g. `employee`) or `NONE` to signify no relation

relation extraction can be formulated as a classification task by creating an representation of the mentions $m_1$ and $m_2$ in their context ($d$) as input and the relation type (or `NONE`) as output. (Note that this assumes at most one relation type holds per entity pair.)

In previous state-of-the-art approaches, considerable effort was invested into creating representations of the mentions in context for ML methods, frequently involving e.g. carefully engineered representations of dependency paths.

Fortunately for us, with recent Transformer-based approaches such as BERT, these representations can be simplified into marking the entities in the original text in some way, and providing the text with marked entities to the model as input.

For example, given

* Document text $d$ = `Bill Gates and Paul Allen founded Microsoft`
* Mentions $m_1$ = `(0, 10, PERSON, Bill Gates)` and $m_2$ = `(34, 43, ORG, Microsoft)`
* Relation type `founder`

We could formulate the classification example e.g. as

* input: `PERSON and Paul Allen founded ORGANIZATION`
* output: `founder`

Here using the literal type strings `PERSON` and `ORGANIZATION` to mark the two mentions under consideration. (Note that other mentions in context are not marked.)

## Project work

As the course project involves implementing a relation extraction system along the lines outlined above, we will not provide an implementation here. You may find the previously covered notebooks on text classification helpful as references for your implementation.

(We will provide support for annotating training and evaluation data for your project work.)